In [58]:
import wandb
import os
from datasets import Dataset, load_from_disk, concatenate_datasets
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from transformers import BertModel, BertTokenizerFast, AutoModelForSequenceClassification
import torch
import numpy as np
import pickle as pkl

from sklearn.decomposition import PCA

from copy import deepcopy

In [38]:
top_n = 500
load_top_n = True

In [39]:
path_to_finetuned_bert = '/home/ubuntu/partisan_bias_detection/best_models/final_models/final_bert'
path_to_pretrained_bert = 'bert-base-uncased'

In [40]:
CWD = os.getcwd()
DATASET_DIR = os.path.join(CWD, 'data')

dataset = load_from_disk(DATASET_DIR)
dataset = dataset.rename_column('label', 'labels')

# ds = concatenate_datasets([dataset['train'], dataset['val'], dataset['test']]).shuffle(seed=100).select(range(500000))
ds = dataset['train']
ds = ds.remove_columns('input_ids')
ds = ds.to_pandas()

left = Dataset.from_pandas(ds.loc[ds['labels'] == 4])
leftcenter = Dataset.from_pandas(ds.loc[ds['labels'] == 3])
neutral = Dataset.from_pandas(ds.loc[ds['labels'] == 2])
rightcenter = Dataset.from_pandas(ds.loc[ds['labels'] == 1])
right = Dataset.from_pandas(ds.loc[ds['labels'] == 0])

/home/ubuntu/.local/lib/python3.8/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [41]:
if load_top_n:
    with open(f'top_{top_n}_words.pkl', 'rb') as f:
        top_words = pkl.load(f)
else:
    vec = TfidfVectorizer()

    print('fitting left')
    left_scores = vec.fit_transform(left['text'])
    top_left = sorted(list(zip(vec.get_feature_names(), left_scores.sum(0).getA1())), key=lambda x: x[1], reverse=True)[:top_n]
    top_left = [word for word, score in top_left]

    print('fitting leftcenter')
    leftcenter_scores = vec.fit_transform(leftcenter['text'])
    top_leftcenter = sorted(list(zip(vec.get_feature_names(), leftcenter_scores.sum(0).getA1())), key=lambda x: x[1], reverse=True)[:top_n]
    top_leftcenter = [word for word, score in top_leftcenter]

    print('fitting neutral')
    neutral_scores = vec.fit_transform(neutral['text'])
    top_neutral = sorted(list(zip(vec.get_feature_names(), neutral_scores.sum(0).getA1())), key=lambda x: x[1], reverse=True)[:top_n]
    top_neutral = [word for word, score in top_neutral]

    print('rightcenter neutral')
    rightcenter_scores = vec.fit_transform(rightcenter['text'])
    top_rightcenter = sorted(list(zip(vec.get_feature_names(), rightcenter_scores.sum(0).getA1())), key=lambda x: x[1], reverse=True)[:top_n]
    top_rightcenter = [word for word, score in top_rightcenter]

    print('fitting right')
    right_scores = vec.fit_transform(right['text'])
    top_right = sorted(list(zip(vec.get_feature_names(), right_scores.sum(0).getA1())), key=lambda x: x[1], reverse=True)[:top_n]
    top_right = [word for word, score in top_right]
    
    common = list(
            set(top_left).intersection(set(top_leftcenter), set(top_neutral), set(top_rightcenter), set(top_right)).union(
                set(top_left).intersection(set(top_leftcenter)).union(
                set(top_left).intersection(set(top_neutral)).union(
                set(top_left).intersection(set(top_rightcenter)).union(
                set(top_left).intersection(set(top_right)).union(
                    set(top_leftcenter).intersection(set(top_neutral)).union(
                    set(top_leftcenter).intersection(set(top_rightcenter)).union(
                    set(top_leftcenter).intersection(set(top_right)).union(
                        set(top_neutral).intersection(set(top_rightcenter)).union(
                        set(top_neutral).intersection(set(top_right)).union(
                            set(top_rightcenter).intersection(set(top_right))
                        )
                        )
                    )
                    )
                    )
                )
                )
                )
                )
            )
        )

    # print(f'common: \n', common)

    for word in common:
        if word in top_left:
            top_left.remove(word)
        if word in top_leftcenter:
            top_leftcenter.remove(word)
        if word in top_neutral:
            top_neutral.remove(word)
        if word in top_rightcenter:
            top_rightcenter.remove(word)
        if word in top_right:
            top_right.remove(word)

    print(f'left top {top_n}: \n', top_left)
    print(f'left-center top {top_n}: \n', top_leftcenter)
    print(f'neutral top {top_n}: \n', top_neutral)
    print(f'right-center top {top_n}: \n', top_rightcenter)
    print(f'right top {top_n}: \n', top_right)

    top_words = {'left': top_left, 
                 'left_center': top_leftcenter, 
                 'neutral': top_neutral, 
                 'right_center': top_rightcenter, 
                 'right': top_right}

    with open(f'top_{top_n}_words.pkl', 'wb') as f:
        pkl.dump(top_words, f)

print(f'top {top_n}')
for bias, words in top_words.items():
    print(f'{bias}: {words}\n')

top 500
left: ['bush', 'climate', 'israeli', 'romney', 'movement', 'gop', 'let', 'actually', 'afghanistan', 'course', 'weapons', 'politics', 'thing', 'labor', 'ever', 'class', 'nothing', 'book', 'mccain', 'middle', 'question', 'conservative', 'palestinian', 'civil', 'corporate', 'intelligence', 'army', 'abortion', 'speech', 'peace', 'debate', 'action', 'iraqi', 'candidate', 'democracy', 'kind', 'watch', 'lt', 'legal', 'clear', 'sanders', 'private', 'almost', 'isn', 'given', 'george', 'gt', 'troops', 'having', 'seems', 'hard', 'done', 'rather', 'via', 'mother', 'whose', 'paul', 'reform', 'means', 'instead', 'face', 'stop', 'evidence', 'talk', 'himself', 'bad', 'words', 'immigration']

left_center: ['baptist', 'globalpost', 'god', 'lgbt', 'chicago', 'religious', 'teachers', 'study', 'pastor', 'churches', 'virginia', 'india', 'brown', 'southern', 'ministry', 'christian', 'love', 'music', 'known', 'chinese', 'parents', 'ukraine', 'convention', 'faith', 'research', 'bbc', 'baptists', 'stude

In [53]:
def get_embeds(model, all_words, specific_list, pca_dim=50):
    tok = BertTokenizerFast.from_pretrained('bert-base-uncased')
    embeds = []
    
    biases = list(all_words.keys())
    is_specific = []
    for bias in biases:
        words = all_words[bias]
        is_specific += [f'{bias}_specific' if word in specific_list else bias for word in words]
        model = model.to('cuda:0')
        input_ = torch.tensor(tok.convert_tokens_to_ids(words), dtype=torch.int).unsqueeze(0).to('cuda:0')
        embeds.extend(torch.nn.functional.pad(model(input_).last_hidden_state.squeeze(), pad=(0, 2)).tolist())
    
    if pca_dim:
        pca = PCA(n_components = pca_dim, random_state = 100)
        pca_embeds = pca.fit_transform(embeds)
        pca_embeds = pca_embeds.tolist()
        final_embeds = deepcopy(pca_embeds)
    else:
        final_embeds = embeds
        
    class2bias = {i: biases[i] for i in range(len(biases))}
    
    lengths = [len(all_words[bias]) for bias in biases]
    lengths.insert(0, 0) 
    lengths = np.cumsum(lengths)
    top = [all_words[bias] for bias in biases]

    j = 0

    for i in range(len(final_embeds)):
        if i == lengths[j + 1]:
            j += 1
        final_embeds[i].append(class2bias[j])
        final_embeds[i].append(top[j][i - lengths[j]])
        final_embeds[i].append(is_specific[i])
    
    return final_embeds
    
    
def visualize_embeds(final_embeds, specific=[], notes=''):
    columns = [f'D{i}' for i in range(len(list(final_embeds.values())[0][0]) - 3)]
    columns.append('bias')
    columns.append('word')
    columns.append('specific')
    
    wandb.init(project=f'top_embeds_{"_".join(list(final_embeds.keys()))}', notes=notes)
    for model_type, embeds in final_embeds.items():
        final_embeds[model_type] = wandb.Table(
            columns = columns,
            data = embeds
        )
        
    wandb.log(final_embeds)
    wandb.finish()    
    
    
def get_embeds_and_visualize(words, specific=[], pretrained=None, finetuned=None, hat=None, pca_dim=50, notes=''):
    final_embeds = {}
    if pretrained:
        final_embeds['pretrained'] = get_embeds(pretrained, words, specific, pca_dim)

    if finetuned: 
        final_embeds['finetuned'] = get_embeds(finetuned, words, specific, pca_dim)
        
    if hat: 
        final_embeds['hat'] = get_embeds(hat, words, specific, pca_dim)
        
    visualize_embeds(final_embeds, notes)

In [66]:
pretrained_model = BertModel.from_pretrained(path_to_pretrained_bert)
finetuned_model = BertModel.from_pretrained(path_to_finetuned_bert)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /home/ubuntu/partisan_bias_detection/best_models/final_models/final_bert were not used when initializing BertModel:

In [ ]:
# Common left vs. right words
specific_words = {
    'left': ['rally', 'demonstrations', 'demonstrators', 'protesters', 'termination', 'asylum', 'undocumented'], 
    'right': ['riots', 'anarchists', 'mob', 'rioters', 'infanticide', 'illegal', 'aliens']
}

new_words = {}
specific_list = []
for bias, words in top_words.items():
    new_words[bias] = top_words[bias] + specific_words.get(bias, [])
    specific_list += specific_words.get(bias, [])

get_embeds_and_visualize(new_words, specific_list, pretrained_model, finetuned_model, pca_dim=None, notes='plotting specific words')